In [4]:
# just installing napari, geopandas and tifffile should be enough to import everything
import pandas as pd
import napari
from tifffile.tifffile import imread
import geopandas as gp
from shapely.geometry import Polygon

In [10]:
def crop(roi_df, image):
    # get last 2 columns, this is y, x
    bbox = roi_df.iloc[:,-2:].to_numpy()
    return image[:, int(bbox[0,0]):int(bbox[2,0]), int(bbox[0,1]): int(bbox[2,1])]

def crop_mask(roi_df, mask):
    # get last 2 columns, this is y, x
    bbox = roi_df.iloc[:,-2:].to_numpy()
    return mask[int(bbox[0,0]):int(bbox[2,0]), int(bbox[0,1]): int(bbox[2,1])]

def crop_coords(roi_df, points):
    """Subtract the bbox minimum x and y coordinates to adjust point locations"""
    min_x = roi.iloc[0,-1]
    min_y = roi.iloc[0,-2]
    points.loc[:,'x'] -= min_x
    points.loc[:,'y'] -= min_y
    return points

def plot_points_napari(image,points_data,roi,mask,genes_of_interest,color_palette=["green","blue"] ,pt_size=1):
    # Just loading everything and converting to geopandas.
    points_data.columns = ['x','y','cell_id','gene_target']
    points_data = points_data[points_data.gene_target.isin(genes_of_interest)]
    gdf = gp.GeoDataFrame(
        points_data, geometry=gp.points_from_xy(points_data.x, points_data.y)
    )
    
    # Crop based on roi coordinates
    image_crop =crop(roi, image)
    # Loading the polygon in x and y since that is how you gave the data, but napari saves in y, x order unless you rearranged dim order
    #
    polygon= Polygon(roi.iloc[:,:-3:-1].to_numpy())
    poly_gpd=gp.GeoDataFrame(index=[0],geometry=[polygon])

    cropped_mask = crop_mask(roi, mask)

    # Basically fastest way to get all points within a polygon.
    subset_points=gp.sjoin(gdf, poly_gpd, predicate='within')
    subset_points=crop_coords(roi,subset_points)
    subset_points['gene_target'] = pd.Categorical(subset_points['gene_target'])
    # We use the gene target code, which is an integer as for the color cycle it is not accepted to have a string. However, with text we can still see the gene target
    point_props = {'cell_id': subset_points['cell_id'].to_numpy(), 
                'gene_target': subset_points['gene_target'].to_numpy()}
    point_props['gene_target'] = pd.Categorical(point_props['gene_target'])
    
    viewer = napari.Viewer()
    viewer.add_image(image_crop)
    viewer.add_image(cropped_mask)
    points_layer=viewer.add_points(subset_points[['y','x']].to_numpy(), 
                                properties = point_props,
                                face_color = 'gene_target',
                                face_color_cycle = color_palette,
                                size=pt_size,
                                edge_width_is_relative=False)

In [11]:
image = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/preprocess/sample_2d_r1_s1.stack.clahe.tiff")
points_data = pd.read_csv('/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/dedup_spots/sample_2d_r1_s1.spots_markedDups.txt', delimiter='\t', header=None)
roi = pd.read_csv('./roi2.csv')

plot_points_napari(image,points_data,roi,["Pln","Pecam1"], color_palette= ["green","red"] ,pt_size=5)

TypeError: plot_points_napari() missing 1 required positional argument: 'genes_of_interest'

In [12]:
## Load a labeled mask
mask = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_MolCart_results/retained_masks/sample_2d_r1_s1_cellpose.retained_masks.tif")
mask= mask.astype('int32')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/florian_wuennemann/1_Projects/MI_project/data/nf_MolCart_results/retained_masks/sample_2d_r1_s1_cellpose.retained_masks.tif'

In [13]:
mask_crop =crop_mask(roi, mask)
image_crop =crop(roi, image)

NameError: name 'mask' is not defined

## Write a larger view of endocardium with infiltration and Ccr2 from an image at d2

###  Example 1: sample_2d_r1_s1.stack.clahe.tiff

In [15]:
roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop.csv")
image = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/preprocess/sample_2d_r1_s1.stack.clahe.tiff")
## Load a labeled mask
mask = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/retained_masks/sample_2d_r1_s1_cellpose.retained_masks.tif")
mask= mask.astype('int32')
mask_crop =crop_mask(roi, mask)

points_data = pd.read_csv('/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/dedup_spots/sample_2d_r1_s1.spots_markedDups.txt', delimiter='\t', header=None)

#roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop2.csv")
#roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop3.csv")


plot_points_napari(image,points_data,roi,mask,["Ccr2","Nppa","Npr3","Cd74"], color_palette= ["magenta","yellow","cyan","red"] ,pt_size=20)
# viewer = napari.Viewer()
# viewer.add_image(image)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop.csv'

###  Example 2: sample_2d_r2_s1.stack.clahe.tiff

In [17]:
image = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/preprocess/sample_2d_r2_s1.stack.clahe.tiff")
## Load a labeled mask
mask = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/retained_masks/sample_2d_r2_s1_cellpose.retained_masks.tif")
mask= mask.astype('int32')
mask_crop =crop_mask(roi, mask)

points_data = pd.read_csv('/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/dedup_spots/sample_2d_r2_s1.spots_markedDups.txt', delimiter='\t', header=None)
roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_r2_s1_ccr2_infiltration_crop1.csv")
#roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop2.csv")
#roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop3.csv")

plot_points_napari(image,points_data,roi,["Ccr2"], color_palette= ["magenta"] ,pt_size=20)
# viewer = napari.Viewer()
# viewer.add_image(image)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_r2_s1_ccr2_infiltration_crop1.csv'

# Example 3 

In [19]:
image = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/preprocess/sample_2d_r1_s2.stack.clahe.tiff")

points_data = pd.read_csv('/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/dedup_spots/sample_2d_r1_s2.spots_markedDups.txt', delimiter='\t', header=None)
roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_r1_s2_ccr2_infiltration_crop1.csv")
#roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop2.csv")
#roi = pd.read_csv("/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_ccr2_infiltration_crop3.csv")

plot_points_napari(image,points_data,roi,["Nppa"], color_palette= ["magenta"] ,pt_size=20)
#viewer = napari.Viewer()
#viewer.add_image(image)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/florian_wuennemann/2_Areas/conferences/2023_Spatial_Biology_congress_Rotterdam/napari_images/sample_2d_r1_s2_ccr2_infiltration_crop1.csv'

In [20]:
image = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/preprocess/sample_2d_r1_s1.stack.clahe.tiff")
points_data = pd.read_csv('/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/dedup_spots/sample_2d_r1_s1.spots_markedDups.txt', delimiter='\t', header=None)
roi = pd.read_csv('./roi.csv')

plot_points_napari(image,points_data,roi,["Ccr2","Pecam1"], color_palette= ["green","red"] ,pt_size=5)

TypeError: plot_points_napari() missing 1 required positional argument: 'genes_of_interest'

In [21]:
viewer = napari.Viewer()
mask_crop = crop_mask(roi, mask)
image_crop = crop(roi, image)
viewer.add_image(image_crop)
viewer.add_image(mask_crop)

<Image layer 'mask_crop' at 0x168e92e10>

In [23]:
image = imread("/Users/florian_wuennemann/1_Projects/MI_project/data/nf_molkart_results/preprocess/sample_2d_r1_s2.stack.clahe.tiff")
viewer = napari.Viewer()
viewer.add_image(image)

<Image layer 'image' at 0x168e4c8d0>

: 